In [3]:
# ngram
def ngram(sentence, n):
    chunk = []
    for i in xrange(0,len(sentence.decode('utf-8'))-n+1):
        chunk.append(sentence.decode('utf-8')[i:i+n])
    return chunk


# segmentWord(try cut string in two segments)
def segment(sentence):
    n=len(sentence.decode('utf-8'))
    segments=[]
    for i in xrange(0,n):
        seg1=sentence.decode('utf-8')[0:i]
        seg2=sentence.decode('utf-8')[i:n+1]
        segments.append([seg1,seg2])
    return(segments)
        
    
# word_freq
def word_freq(sentences=None,n=6):     #senteces 為包含多個sentence的list
    pieces=[]
    for i in xrange(1,n+1):
        for sentence in sentences:
            pieces.append(ngram(sentence,i))
    pieces_concat=[]
    for piece in pieces:
        for p in piece:
            pieces_concat.append(p)
    pieces_freq={}
    for pf in pieces_concat:
        pieces_freq[pf]=0
    for pf in pieces_concat:
        if pieces_freq[pf]>0:
            pieces_freq[pf]+=1
        else:
            pieces_freq[pf]=1
    return pieces_freq
        

# word_prob
def word_prob(sentences=None,n=6):
    pieces=[]
    for i in xrange(1,n+1):
        for sentence in sentences:
            pieces.append(ngram(sentence,i))
    pieces_concat=[]
    for piece in pieces:
        for p in piece:
            pieces_concat.append(p)
    pieces_freq={}
    for pf in pieces_concat:
        pieces_freq[pf]=0
    for pf in pieces_concat:
        if pieces_freq[pf]>0:
            pieces_freq[pf]+=1
        else:
            pieces_freq[pf]=1
    word_concat=""
    for sentence in sentences:
        word_concat=word_concat+senctence
    N=len(word_concat.decode('utf-8'))
    pieces_prob={}
    for pf in pieces_freq:
        pieces_prob[pf]=pieces_freq[pf]/N
    return pieces_prob

    
# cohesion  算每一個詞再斷詞後機率是否跟該詞獨立，一個字的會自動過濾掉
def word_isolation(sentences,iso=0,n=6):  #iso表示想選擇isolation value >多少的數值
    import math
    word_group=word_freq(sentences,n)
    word_concat=""
    for sentence in sentences:
        word_concat=word_concat+sentence
    N=len(word_concat.decode('utf-8'))
    word_iso={}
    for word in word_group:
        word_mi=[]
        wordseg=segment(str(word.encode('utf-8')))
        for wseg in wordseg:
            try:
                word_f = word_group[word]
                wseg1_f = word_group[wseg[0]]
                wseg2_f = word_group[wseg[1]]
                mi=math.log(N,2) + math.log(word_f,2) - math.log(wseg1_f,2) - math.log(wseg2_f,2)
                word_mi.append(mi)
            except:
                pass
        try:
            word_iso[word]=min(word_mi)
        except:
            pass
    return dict((key,value) for key, value in word_iso.iteritems() if value >=iso)



# disorder
def word_disorder(sentences,n=6,iso=0,d=1):    #val為isolation value, n為ngram長度, d為亂度閥值
    #from itertools import chain
    import re
    import numpy
    import math
    pieces={}
    for i in xrange(1,n+1):
        c=[]
        for sentence in sentences:
            c=c+ngram(sentence,i)
        pieces[i]=c

    words_iso = word_isolation(sentences,iso,n)
    words_freq = word_freq(sentences,n)
    words_dis={}
    i=0
    for word in words_iso:
        i+=1
        try:
            N=len(word)
            if N<n:
                base=pieces[N+1]

                match1=[]
                match2=[]
                for b in base:

                    if bool(re.search("^"+word,b)):
                        match1.append(b)
                    if bool(re.search(word+"$",b)):
                        match2.append(b)
                freq1=[]
                for m in set(match1):
                    freq1.append(words_freq[m])

                SUM=sum(freq1)
                log_a=[-(math.log(float(a)/SUM,2)) for a in freq1]
                pre=numpy.mean(log_a)

                freq2=[]
                for m in set(match2):
                    freq2.append(words_freq[m])

                SUM2=sum(freq2)
                log_a2=[-(math.log(float(a)/SUM,2)) for a in freq2]
                post=numpy.mean(log_a2)

                if math.isnan(pre)|math.isnan(post):
                    if math.isnan(pre):
                        words_dis[word]=post
                    else:
                        words_dis[word]=pre
                else:
                    words_dis[word]=min([pre,post]) 
            else:
                pass
        except:
            pass
    return dict((key,value) for key, value in words_dis.iteritems() if value >=d)




def remove_punc(sentences):
    import string 
    import re
    delEStr = '['+string.punctuation + u' '+']'
    delCStr = u'[▲►★《》（）()&%￥#@$！?,.><*{}\'`"【】　、。，：；”～‘＠＃＄％︿＆＊｀＝－＋｜／＼？『』」「\/]'
    s=[]
    for sentence in sentences:
        if type(sentence)==unicode:
            sentence=re.sub(delEStr,u'',sentence)
            sentence=re.sub(delCStr,u'',sentence)
            s.append(sentence.encode('utf-8'))
        else:
            sentence=re.sub(delEStr,u'',sentence.decode('utf-8'))
            sentence=re.sub(delCStr,u'',sentence)
            s.append(sentence.encode('utf-8'))
    return s  


def replace_punc(sentences):
    import string 
    import re
    delEStr = '['+string.punctuation + u' '+']'
    delCStr = u'[▲►★《》（）()&%￥#@$！?,.><*{}\'`"【】　、。，：；”～‘＠＃＄％︿＆＊｀＝－＋｜／＼？『』」「\/]'
    s=[]
    for sentence in sentences:
        if type(sentence)==unicode:
            sentence=re.sub(delEStr,u',',sentence)
            sentence=re.sub(delCStr,u',',sentence).encode('utf-8')
            sentence=sentence.split(",")
            #s.append(sentence.encode('utf-8'))
            s.append(sentence)
        else:
            sentence=re.sub(delEStr,u',',sentence.decode('utf-8'))
            sentence=re.sub(delCStr,u',',sentence).encode('utf-8')
            sentence=sentence.split(",")
            #s.append(sentence.encode('utf-8'))
            s.append(sentence)
    return s[0]

In [6]:
text=['''行政院郝院長二十八日召開相關部會會議，聽取主計處報告「八十二年度國家總供需計畫」，原則確定八十二年度中央政府總預算規模將首度突破新台幣一兆元大關，達一兆一千一百億元，較八十一年度預算九千八百一十二億元約增加百分之十三。由於收支短絀擴大，下年度課稅收入祇有五千四百多億元，和今年相較沒有成長；國營事業盈餘只有五百億元左右，和今年相較反而減少三百億元，其全部收入為六千九百億元，預計發行建設公債三千五百億元，為歷年公債發行額最高紀錄；同時，下年度償還歷年公債本息金額超過一千億元，佔總預算支出百分之十。又據估計：下年度平均國民所得亦將突破一萬美元大關，達一萬零一百九十美元，經濟成長率為百分之七點○三。<p>基於此，下年度中央政府總預算，係一持續擴大性的赤字預算，值此國際經濟景氣想翻身不得等待期間，區域經濟結盟日益強化或深化，以及國內民間投資意願低落之際，下年度政府預算持續擴大公共投資，藉以增加國內有效需求，並帶動經濟繼續成長和穩定，自為當前經濟發展所必需，亦為推動國建六年計畫有效配合。惟值得注意的，仍有下列數點：<p>第一、預算赤字增加太快。按七十八年度預算赤字為一千三百八十九億元，七十九年度為一千五百七十億元，八十年度為一千七百七十二億元，八十一年度為三千一百億元，至八十二年度則高達四千二百億元，佔總預算超過三分之一。此固由於近年來經濟景氣尚未翻身，投資意願依然低落，致使國家稅收沒有成長，未能配合公共投資支出的增加。但從另一方面來看，亦有由於既得利益者透過政治管道，而獲得多項稅課減免優待，乃造成稅基日益縮小；同時，更由於部分銳率偏高，或重複課稅，形成地下經濟猖獗，逃漏稅捐乃至利益輸送層出不窮，影響課稅公平，至深且鉅。因此，改革稅制，修正稅法，加強稽徵，杜絕逃漏，為今後亟待努力的取向。<p>第二、公共工程落後甚多，近年來由於受到社會變動，非經濟因素之影響，政府各項重大公共投資支出預算，皆未能按照預訂進度順利執行，乃產生工程落後之現象，亦由此，國庫資金累積甚鉅，無法消化。根據報導，國庫現金結存高達千億元以上，實為國家資源的浪費。八十二年度公共投資繼續增加，并大力推動國建六年計劃，若沒有突破性的執行辦法及監督制度，則國庫結存資金恐怕亦將隨著中央總預算的增加而增加。因此，我們期望：各有關執行單位應迅速放棄本位主義，共同研擬多元化因應措施，相互聯繫追蹤監督辦法，以克服當前公共投資的困難，進而達成公共工程預期進度之目標?<p>第三、對公債依賴性太高。因預算赤字繼續增加而大量發行公債，以資挹注，自不可避免，但是對公債之運用，亦須妥為規劃。按八十二年度政府預算將發行建設公債三千五百億元，佔中央總預算比例，高達百分之三十以上。因此，為減輕公債衝擊或避免通貨膨脹，我們期望在預算中將經常帳與資本帳，必須加以界定劃分，若屬經常項目，如一般行政支出，其財源似不宜仰賴發行公債，而盡可能由稅收充之；若屬資本項目，如鐵、公路及捷運系統等具自償性者，則其財源不宜仰賴稅收，而由公債收入充之。同時，我們建議：凡是完全可以自償的公共投資，不妨考慮劃出開放民營，如此不但可以減少公債大量發行對社會經濟的衝擊，並且亦可使國建六年計畫的進行更具效率。第四、高所得不代表就有高品質生活。按下年度國民所得預估高達一萬美元，假如我國文化道德水準沒有相對配合提高，不免令人擔憂，到時候社會現象更加紛亂，政治問題亦趨嚴重。高所得的成長並不代表高品質的精神生活。相反的，空有許多財富，而道德修養無法相對提高，衹會喪失人性，使社會淪於更黑暗的一面。因此，我們期望：應加強建設文化、提高道德水準。換句話說，應增加文化建設經費，如對教育、文化、科學支出應大幅增加比重，若硬體建設投資為百分之二十，則軟體建設投資亦得百分之二十。唯有如此相互配合，才能加速工業升級，並提高生活品質。<p>總之，八十二年度中央政府總預算將突破一兆元大關，是為因應經濟景氣，並維持經濟持續成長，而傾向於擴大性的赤字預算，惟為配合推動國建六年計畫，我們期望：要消除非經濟因素，以免阻礙公共投資；要兼顧硬體與軟體建設之投資比重，以提高生活品質；要開放民營參加建設，以減少公債發行之依賴性；要擴大稅基，降低稅率，加強稽徵，以保持課稅的公平；最後要採行複式預算制度，以發揮預算執行與監督之功能。''']
#text=["巴西當局於29日宣布，未來變性受刑人可以自由選擇要到男或女監獄服刑巴西我巴西可以巴西可巴西"]
#text=["ddd","ddddddd","aaaa"]
#text=remove_punc(text)
text=replace_punc(text)

#print type(text), text, len(text)


dis=word_disorder(text,n=8,iso=0,d=0.5)

for w in dis:
    print w, dis[w]

print "======================"

未能 1.0
國內 1.0
軟體建設 1.0
十二 1.20751874964
配合 1.58496250072
近年來 1.0
自償 1.0
經濟 3.29774444509
一萬 1.08496250072
經濟景氣 1.0
公共 1.26624754041
發行 2.40735492206
體建設 1.08496250072
沒有 1.66666666667
建設 2.29036242191
政治 1.0
年度政府預算 1.0
增加而 1.0
收入 1.58496250072
政府 1.29248125036
年度國 1.0
工程 1.08496250072
不代表 1.0
年度預算 1.0
執行 1.58496250072
不宜仰賴 1.0
亦將 1.0
相互 1.0
赤字 1.02944684453
體建設投資 1.0
一百 1.08496250072
大量發行 1.0
減少 1.08496250072
中央 1.02944684453
監督 1.58496250072
道德 1.08496250072
國建六年計 1.20751874964
有效 1.0
硬體 1.0
八十二年度 2.38496250072
公共投資 1.66666666667
三百 1.0
結存 1.0
當前 1.0
預算將 1.0
進度 1.0
使國 1.0
投資 2.21808627093
百分之 2.08496250072
八十一年度 1.0
五百 1.20751874964
中央總預算 1.0
加強 1.08496250072
對公債 1.0
行政 1.0
達一萬 1.0
十九 1.0
高達 2.0
預算赤字 1.58496250072
相對 1.0
可以 1.0
歷年公債 1.0
無法 1.0
國建六年計畫 0.5
增加 2.55735492206
擴大 1.66666666667
繼續 1.08496250072
稅收 1.58496250072
國家 1.58496250072
若屬 1.0
超過 1.0
七十 1.5
以減少公債 1.0
其財源 1.0
公共工程 1.0
分之 1.5148736717
將突破一 1.0
年度為一千 1.0
性的 1.08496250072
支出 1.75
社會 2.0
和今年相較 1.0
我們 1.32192809489
一兆 1.08

In [5]:
# disorder version 2
import time
def word_disorder(sentences,n=6,iso=0,d=1):    #val為isolation value, n為ngram長度, d為亂度閥值
    #from itertools import chain
    import re
    import numpy
    import math
    pieces={}
    for i in xrange(1,n+1):
        c=[]
        for sentence in sentences:
            c=c+ngram(sentence,i)
        pieces[i]=c

    #words_iso = word_isolation(sentences,iso,n)
    words_freq = word_freq(sentences,n)
    words_dis={}
    i=0
    for word in words_freq:
        i+=1
        try:
            N=len(word)
            if N>1 & N<n:
                base=pieces[N+1]

                match1=[]
                match2=[]
                for b in base:

                    if bool(re.search("^"+word,b)):
                        match1.append(b)
                    if bool(re.search(word+"$",b)):
                        match2.append(b)
                freq1=[]
                for m in set(match1):
                    freq1.append(words_freq[m])

                SUM=sum(freq1)
                log_a=[-(math.log(float(a)/SUM,2)) for a in freq1]
                pre=numpy.mean(log_a)

                freq2=[]
                for m in set(match2):
                    freq2.append(words_freq[m])

                SUM2=sum(freq2)
                log_a2=[-(math.log(float(a)/SUM,2)) for a in freq2]
                post=numpy.mean(log_a2)

                if math.isnan(pre)|math.isnan(post):
                    if math.isnan(pre):
                        words_dis[word]=post
                    else:
                        words_dis[word]=pre
                else:
                    words_dis[word]=min([pre,post]) 
            else:
                pass
        except:
            pass
    return dict((key,value) for key, value in words_dis.iteritems() if value >=d)

text=['''行政院郝院長二十八日召開相關部會會議，聽取主計處報告「八十二年度國家總供需計畫」，原則確定八十二年度中央政府總預算規模將首度突破新台幣一兆元大關，達一兆一千一百億元，較八十一年度預算九千八百一十二億元約增加百分之十三。由於收支短絀擴大，下年度課稅收入祇有五千四百多億元，和今年相較沒有成長；國營事業盈餘只有五百億元左右，和今年相較反而減少三百億元，其全部收入為六千九百億元，預計發行建設公債三千五百億元，為歷年公債發行額最高紀錄；同時，下年度償還歷年公債本息金額超過一千億元，佔總預算支出百分之十。又據估計：下年度平均國民所得亦將突破一萬美元大關，達一萬零一百九十美元，經濟成長率為百分之七點○三。<p>基於此，下年度中央政府總預算，係一持續擴大性的赤字預算，值此國際經濟景氣想翻身不得等待期間，區域經濟結盟日益強化或深化，以及國內民間投資意願低落之際，下年度政府預算持續擴大公共投資，藉以增加國內有效需求，並帶動經濟繼續成長和穩定，自為當前經濟發展所必需，亦為推動國建六年計畫有效配合。惟值得注意的，仍有下列數點：<p>第一、預算赤字增加太快。按七十八年度預算赤字為一千三百八十九億元，七十九年度為一千五百七十億元，八十年度為一千七百七十二億元，八十一年度為三千一百億元，至八十二年度則高達四千二百億元，佔總預算超過三分之一。此固由於近年來經濟景氣尚未翻身，投資意願依然低落，致使國家稅收沒有成長，未能配合公共投資支出的增加。但從另一方面來看，亦有由於既得利益者透過政治管道，而獲得多項稅課減免優待，乃造成稅基日益縮小；同時，更由於部分銳率偏高，或重複課稅，形成地下經濟猖獗，逃漏稅捐乃至利益輸送層出不窮，影響課稅公平，至深且鉅。因此，改革稅制，修正稅法，加強稽徵，杜絕逃漏，為今後亟待努力的取向。<p>第二、公共工程落後甚多，近年來由於受到社會變動，非經濟因素之影響，政府各項重大公共投資支出預算，皆未能按照預訂進度順利執行，乃產生工程落後之現象，亦由此，國庫資金累積甚鉅，無法消化。根據報導，國庫現金結存高達千億元以上，實為國家資源的浪費。八十二年度公共投資繼續增加，并大力推動國建六年計劃，若沒有突破性的執行辦法及監督制度，則國庫結存資金恐怕亦將隨著中央總預算的增加而增加。因此，我們期望：各有關執行單位應迅速放棄本位主義，共同研擬多元化因應措施，相互聯繫追蹤監督辦法，以克服當前公共投資的困難，進而達成公共工程預期進度之目標?<p>第三、對公債依賴性太高。因預算赤字繼續增加而大量發行公債，以資挹注，自不可避免，但是對公債之運用，亦須妥為規劃。按八十二年度政府預算將發行建設公債三千五百億元，佔中央總預算比例，高達百分之三十以上。因此，為減輕公債衝擊或避免通貨膨脹，我們期望在預算中將經常帳與資本帳，必須加以界定劃分，若屬經常項目，如一般行政支出，其財源似不宜仰賴發行公債，而盡可能由稅收充之；若屬資本項目，如鐵、公路及捷運系統等具自償性者，則其財源不宜仰賴稅收，而由公債收入充之。同時，我們建議：凡是完全可以自償的公共投資，不妨考慮劃出開放民營，如此不但可以減少公債大量發行對社會經濟的衝擊，並且亦可使國建六年計畫的進行更具效率。第四、高所得不代表就有高品質生活。按下年度國民所得預估高達一萬美元，假如我國文化道德水準沒有相對配合提高，不免令人擔憂，到時候社會現象更加紛亂，政治問題亦趨嚴重。高所得的成長並不代表高品質的精神生活。相反的，空有許多財富，而道德修養無法相對提高，衹會喪失人性，使社會淪於更黑暗的一面。因此，我們期望：應加強建設文化、提高道德水準。換句話說，應增加文化建設經費，如對教育、文化、科學支出應大幅增加比重，若硬體建設投資為百分之二十，則軟體建設投資亦得百分之二十。唯有如此相互配合，才能加速工業升級，並提高生活品質。<p>總之，八十二年度中央政府總預算將突破一兆元大關，是為因應經濟景氣，並維持經濟持續成長，而傾向於擴大性的赤字預算，惟為配合推動國建六年計畫，我們期望：要消除非經濟因素，以免阻礙公共投資；要兼顧硬體與軟體建設之投資比重，以提高生活品質；要開放民營參加建設，以減少公債發行之依賴性；要擴大稅基，降低稅率，加強稽徵，以保持課稅的公平；最後要採行複式預算制度，以發揮預算執行與監督之功能。''']
#text=["巴西當局於29日宣布，未來變性受刑人可以自由選擇要到男或女監獄服刑巴西我巴西可以巴西可巴西"]
#text=["ddd","ddddddd","aaaa"]
#text=remove_punc(text)

a=time.clock()
text=replace_punc(text)

#print type(text), text, len(text)


dis=word_disorder(text,n=8,iso=6,d=1)

for w in dis:
    print w, dis[w]

print "======================"
print time.clock()-a

課稅 1.58496250072
未能 1.0
減少 1.08496250072
執行 1.58496250072
國內 1.0
近年來 1.0
若屬 1.0
監督 1.58496250072
軟體建設 1.0
道德 1.08496250072
國建六年計 1.20751874964
當前 1.0
公共投資支出 1.0
以減少公債 1.0
硬體 1.0
超過 1.0
體建設投資 1.0
一兆 1.08496250072
其財源 1.0
公共投資 1.66666666667
收入 1.58496250072
公共工程 1.0
國家 1.58496250072
自償 1.0
經濟 3.29774444509
預算 3.2616289376
日益 1.0
提高 1.08496250072
配合 1.58496250072
一萬 1.08496250072
經濟景氣 1.0
不宜仰賴 1.0
八十二年度 2.38496250072
支出 1.75
達一萬 1.0
結存 1.0
有效 1.0
投資 2.21808627093
社會 2.0
體建設 1.08496250072
沒有 1.66666666667
八十一年度 1.0
和今年相較 1.0
文化 1.0
經常 1.0
政治 1.0
成長 1.08496250072
政府 1.29248125036
國庫 1.58496250072
百分之 2.08496250072
中央 1.02944684453
持續 1.08496250072
相互 1.0
國民所得 1.0
中央總預算 1.0
所得 1.0
擴大 1.66666666667
下年度 2.58496250072
分之 1.5148736717
將突破一 1.0
發行 2.40735492206
我們 1.32192809489
總預算 1.38997500048
工程落後 1.0
工程 1.08496250072
相對 1.0
持續擴大 1.0
投資意願 1.0
千五百 1.08496250072
不代表 1.0
歷年公債 1.0
由於 2.32192809489
公債 2.6076423806
利益 1.0
無法 1.0
突破 1.66666666667
建設 2.29036242191
增加 2.55735492206
赤字 1.02944684453
繼續 

In [ ]:
# disorder archive
def word_disorder(sentences,val=10,n=6):    #val為isolation value, n為ngram
    from itertools import chain
    import re
    import numpy
    
    pieces={}
    for i in xrange(1,n+1):
        c=[]
        for sentence in sentences:
            #c = chain.from_iterable([c,ngram(sentence,i)])
            c=c+ngram(sentence,i)
        pieces[i]=c
    words_iso = word_isolation(sentences,val,n)
    
    words_freq = word_freq(sentences,n)
    words_dis={}
    for i in xrange(1,n):
        words_set=dict((key,value) for key, value in words_freq.iteritems() if len(key) == i)
        for word in words_set:
            if i<n:
                base=pieces[i+1]
                match1=[]
                for b in base:
                    if bool(re.search("^"+word,b)):
                        match1.append(b)
                freq1=[]
                for m in match1:
                    freq1.append(words_freq[m])
                SUM=sum(freq1)
                log_a=[-(math.log(float(a)/SUM,2)) for a in freq1]
                pre=numpy.mean(log_a)

                match2=[]
                for b in base:
                    if bool(re.search(word+"$",b)):
                        match2.append(b)
                freq2=[]
                for m in match2:
                    freq2.append(words_freq[m])
                SUM2=sum(freq2)
                log_a2=[-(math.log(float(a)/SUM,2)) for a in freq2]
                post=numpy.mean(log_a2)

                if math.isnan(pre)|math.isnan(post):
                    if math.isnan(pre):
                        words_dis[word]=post
                    else:
                         words_dis[word]=pre
                else:
                    words_dis[word]=min([pre,post])    
            else:
                pass
    return words_dis


In [ ]:
# test disorder
import re
import numpy
import math
val=10
n=6
sentences=text
pieces={}
for i in xrange(1,n+1):
    c=[]
    for sentence in sentences:
        #c = chain.from_iterable([c,ngram(sentence,i)])
        c=c+ngram(sentence,i)
    pieces[i]=c
    #for p in pieces[i]:
    #    print p
    #print "----------"
words_iso = word_isolation(sentences,val,n)

words_freq = word_freq(sentences,n)
#for f in words_freq:
#    print f
words_dis={}
print len(words_freq)
i=0
for word in words_freq:
    i+=1
    print "no." + str(i)
    try:
        N=len(word)
        if N<n:
            base=pieces[N+1]
            print "字長度："+str(len(word))
            print "要比較的字:" , word, "^"+word, word+"$"
            print "被比較的字:"
            match1=[]
            match2=[]
            for b in base:
                print "\t", b
                print "\t","  比較:","^"+word, bool(re.search("^"+word,b))
                print "\t","  比較:","^"+word, bool(re.search(word+"$",b))
                #print "-------"
                if bool(re.search("^"+word,b)):
                    match1.append(b)
                if bool(re.search(word+"$",b)):
                    match2.append(b)
            '''print "match-pre: "
            for m in match1:
                print m
            print "match-post: "
            for m in match2:
                print m'''
                
            
            freq1=[]
            for m in set(match1):
                print "match-pre:",m,words_freq[m]
                freq1.append(words_freq[m])
            print "freq-pre:",freq1
            print "match1", match1
            
            SUM=sum(freq1)
            print "SUM-pre:",SUM
            log_a=[-(math.log(float(a)/SUM,2)) for a in freq1]
            print "log-pre:",log_a
            pre=numpy.mean(log_a)
            print "mean-pre:",pre

            freq2=[]
            for m in set(match2):
                print "match-post:",m,words_freq[m]
                freq2.append(words_freq[m])
            print "freq-post:",freq2

            SUM2=sum(freq2)
            print "SUM-post:",SUM2
            log_a2=[-(math.log(float(a)/SUM,2)) for a in freq2]
            print "log-post:",log_a2
            post=numpy.mean(log_a2)
            print "mean-post:", post
            #mean(-log2(freq1/sum(freq1)))

            if math.isnan(pre)|math.isnan(post):
                if math.isnan(pre):
                    words_dis[word]=post
                    print "亂度:", words_dis[word]
                else:
                    words_dis[word]=pre
                    print "亂度:", words_dis[word]         
            else:
                words_dis[word]=min([pre,post]) 
                print "亂度:", words_dis[word]
        else:
            print '字長度:',len(word)
            pass
        print "========"
    except:
        print "error"
        print "========"
        pass
print words_dis